# SAM3 Image Segmentation for Remote Sensing

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam3_image_segmentation.ipynb)

This notebook demonstrates how to use the Segment Anything Model 3 (SAM3) for segmenting remote sensing images using the `samgeo3` module.

## Installation

First, make sure you have the required dependencies installed:

In [1]:
%pip install "segment-geospatial[samgeo3]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of sam3 to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.9/632.9 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━

## Import Libraries


In [3]:
%pip install --upgrade pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [2]:
%pip install jedi>=0.19

In [3]:
%pip install "segment-geospatial[samgeo3]" --no-deps --no-cache-dir

In [ ]:
import leafmap
from samgeo import SamGeo3, download_file

## Download Sample Data

Let's download a sample satellite image covering the University of California, Berkeley, for testing:


In [ ]:
url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley.tif"
image_path = download_file(url)

In [ ]:
m = leafmap.Map()
m.add_raster(image_path, layer_name="Satellite image")
m

## Request access to SAM3

To use SAM3, you need to request access by filling out this form on Hugging Face: https://huggingface.co/facebook/sam3

Once you have access, uncomment the following code block and run it.

In [ ]:
# from huggingface_hub import login
# login()

## Initialize SAM3

When initializing SAM3, you can choose the backend from "meta", or "transformers".

In [ ]:
sam3 = SamGeo3(backend="meta", device=None, checkpoint_path=None, load_from_HF=True)

## Set the image

You can set the image by either passing the image path or the image URL.

In [ ]:
sam3.set_image(image_path)

## Generate masks with text prompt

In [ ]:
sam3.generate_masks(prompt="building")

## Show the results

In [ ]:
sam3.show_anns()

![annotation](https://github.com/user-attachments/assets/64323223-35a2-4e03-9cee-1b60fa0c12af)

In [ ]:
sam3.show_masks()

## Generate masks by bounding boxes

In [ ]:
# Define boxes in [xmin, ymin, xmax, ymax] format
boxes = [[-122.2597, 37.8709, -122.2587, 37.8717]]

# Optional: specify which boxes are positive/negative prompts
box_labels = [True]  # True=include, False=exclude

# Generate masks
sam3.generate_masks_by_boxes(boxes, box_labels, box_crs="EPSG:4326")

In [ ]:
sam3.show_boxes(boxes, box_labels, box_crs="EPSG:4326")

![bbox](https://github.com/user-attachments/assets/87f0b9ab-a621-4e08-a4de-c90cbf0bf32f)

## Show the results

In [ ]:
sam3.show_anns()

## Save Masks

Save the generated masks to a file. If the input is a GeoTIFF, the output will be a GeoTIFF with the same georeferencing. Otherwise, it will be saved as PNG.

In [ ]:
# Save masks with unique values for each object
# Since the input image is a GeoTIFF, the output will also be a GeoTIFF
sam3.save_masks(output="building_masks.tif", unique=True)

In [ ]:
# Save as binary mask (all foreground pixels are 255)
sam3.save_masks(output="building_masks_binary.tif", unique=False)

## Save Masks with Confidence Scores

You can also save the confidence scores for each mask. The scores indicate the model's confidence for each predicted mask.

In [ ]:
# Save masks and confidence scores
# Each pixel in the scores image will have the confidence value of its mask
sam3.save_masks(
    output="building_masks_with_scores.tif",
    save_scores="building_scores.tif",
    unique=True,
)

In [ ]:
sam3.show_masks(cmap="coolwarm")

![scores](https://github.com/user-attachments/assets/23ec9b07-0de9-4f72-81b2-83a3c499e94e)

### Visualize Confidence Scores

Let's visualize the confidence scores to see which masks have higher confidence:

In [ ]:
m.add_raster("building_masks.tif", layer_name="Building masks", visible=False)
m.add_raster(
    "building_scores.tif",
    layer_name="Building scores",
    cmap="coolwarm",
    opacity=0.8,
    nodata=0,
)
m

![map](https://github.com/user-attachments/assets/fa21320d-b4f3-48f9-a3f2-828f4ed1c567)